### The Goal

iDPGaussianMechanism relies very heavily on the `autodp` library, so I figured I would pull all the autodp code involved in the iDPGaussianMechanism class into one notebook, and see what could be vectorized and sped up, and what couldn't

In [44]:
import numpy as np
from functools import lru_cache
from scipy.optimize import minimize_scalar
import math

In [2]:
## Type hints

# stdlib
from typing import Dict
from typing import List
from typing import Optional
from typing import Iterable
from typing import Tuple

# third party
from nacl.signing import VerifyKey

#### autodp code begins here

In [3]:
def get_eps_ana_gaussian(sigma, delta):
    """ This function calculates the gaussian mechanism given sigma and delta using analytical GM"""
    # Basically inverting the above function by solving a nonlinear equation
    assert(delta >=0 and delta <=1)

    if delta == 0:
        return np.inf
    if np.log(delta) >= get_logdelta_ana_gaussian(sigma, 0.0):
        return 0.0

    def fun(x):
        if x < 0:
            return np.inf
        else:
            return get_logdelta_ana_gaussian(sigma, x) - np.log(delta)

In [4]:
def rdp_to_approxdp(rdp, alpha_max=np.inf, BBGHS_conversion=True):
    # from RDP to approx DP
    # alpha_max is an optional input which sometimes helps avoid numerical issues
    # By default, we are using the RDP to approx-DP conversion due to BBGHS'19's Theorem 21
    # paper: https://arxiv.org/pdf/1905.09982.pdf
    # if you need to use the simpler RDP to approxDP conversion for some reason, turn the flag off

    def approxdp(delta):
        """
        approxdp outputs eps as a function of delta based on rdp calculations
        :param delta:
        :return: the \epsilon with a given delta
        """

        if delta < 0 or delta > 1:
            print("Error! delta is a probability and must be between 0 and 1")
        if delta == 0:
            return rdp(np.inf)
        else:
            def fun(x):  # the input the RDP's \alpha
                if x <= 1:
                    return np.inf
                else:
                    if BBGHS_conversion:
                        return np.maximum(rdp(x) + np.log((x-1)/x)
                                          - (np.log(delta) + np.log(x))/(x-1), 0)
                    else:
                        return np.log(1 / delta) / (x - 1) + rdp(x)

            results = minimize_scalar(fun, method='Brent', bracket=(1,2), bounds=[1, alpha_max])
            if results.success:
                return results.fun
            else:
                # There are cases when certain \delta is not feasible.
                # For example, let p and q be uniform the privacy R.V. is either 0 or \infty and unless all \infty
                # events are taken cared of by \delta, \epsilon cannot be < \infty
                return np.inf
    return approxdp

In [5]:
def approxdp_func_to_approxrdp(eps_func):
    # from an approximate_dp function to approxrdp function
    def approxrdp(alpha, delta):
        rdp = puredp_to_rdp(eps_func(delta))
        return rdp(alpha)

    return approxrdp

In [6]:
def approxdp_func_to_fdp(func):
    """
    from an approxdp function to fdp
    :param func: epsilon as a function of delta by default.
    :param delta_func: if the flag is True, then 'func' is a delta as a function of epsilon.
    :return: fdp function
    """
    #
    # By default, logdelta_func is False, and func is eps as a function of delta
    # fpr = maximize_{delta} approxdp_to_fdp(eps(delta),delta)(fpr)
    # if delta_func is True, it means that 'func' is a delta as a function of eps, then
    # fpr = maximize_{delta} approxdp_to_fdp(eps,delta(eps))(fpr)
    def fdp(fpr):

        assert(0 <= fpr <= 1)
        if fpr == 1:
            return 0

        def fun(eps):
            fdp_eps = approxdp_to_fdp(eps, func(eps))
            fnr = fdp_eps(fpr)
            return -fnr

        results = minimize_scalar(fun, bounds=[0, +np.inf], options={'disp': False})
        if results.success:
            return -results.fun
        else:
            return 0
    return fdp

In [7]:
def pointwise_minimum(f1, f2):
    def min_f1_f2(x):
        return np.minimum(f1(x), f2(x))
    return min_f1_f2

def pointwise_minimum_two_arguments(f1, f2):
    def min_f1_f2(x, y):
        return np.minimum(f1(x, y), f2(x, y))
    return min_f1_f2

def pointwise_maximum(f1, f2):
    def max_f1_f2(x):
        return np.maximum(f1(x), f2(x))
    return max_f1_f2

In [8]:
class Mechanism():
    """
     The base mechanism will use typically two functions to describe the mechanism

    # Attributes (actually functions as well):
    # 1: Approximate DP:   epsilon as a function of delta
    # 2. Renyi DP:   RDP epsilon as a function of \alpha
    # 3. Approximate RDP:  approximate RDP. RDP conditioning on a failure probability delta0.
    # 4. f-DP:  Type II error as a function of Type I error. You can get that from Approximate-DP
    #           or FDP directly.
    # 5. epsilon:  Pure DP bound.  If not infinity, then the mechanism satisfies pure DP.
    # 6. delta0:  Failure probability which documents the delta to use for approximate RDP
    #             in the case when there are no information available about the failure event.

    # If we specify RDP only then it will propagate the RDP calculations to approximate-DP
    # and to f-DP
    # If we specify pure-DP only then it propagates to RDP,  Approximate-DP, f-DP and so on.
    # If we specify approximate-DP only, then it implies an approximate RDP bound with \delta_0.
    # If we specify f-DP only then it propagates to other specifications.
    
    
    # If we specify multiple calculations, then it will take the minimum of all of them
    #      in each category
    """


    def __init__(self):
        # Initialize everything with trivial (non-private) defaults
        def RenyiDP(alpha):
            return np.inf

        def approxRDP(delta, alpha):
            return np.inf

        def approxDP(delta):
            return np.inf

        def fDP(fpr):
            fnr = 0.0
            return fnr

        self.RenyiDP = RenyiDP
        self.approxRDP = approxRDP
        self.approxDP = approxDP
        self.fDP = fDP
        self.eps_pureDP = np.inf

        self.delta0 = np.inf  # indicate the smallest allowable \delta0 in approxRDP that is not inf
        #  We can convert localDP to curator DP by parallel composition and by shuffling.
        
    def get_approxDP(self, delta):
        # Output eps as a function of delta
        return self.approxDP(delta)

    def get_approxRDP(self, delta, alpha):
        # Output eps as a function of delta and alpha
        return self.approxRDP(delta, alpha)

    def get_RDP(self, alpha):
        # Output RDP eps as a function of alpha
        return self.RenyiDP(alpha)

    def get_fDP(self, fpr):
        # Output false negative rate as a function of false positive rate
        return self.fDP(fpr)

    def propagate_updates(self, func, type_of_update,
                          delta0=0,
                          BBGHS_conversion=True,
                          fDP_based_conversion=False):
        # This function receives a new description of the mechanisms and updates all functions
        # based on what is new by calling converters.

        if type_of_update == 'approxDP_func':
            # func outputs eps as a function of delta
            # optional input delta0, telling us from where \epsilon becomes infinity

            self.delta0 = np.minimum(delta0, self.delta0)  # How is this ever anything except 0?!
            self.fDP = pointwise_maximum(self.fDP, approxdp_func_to_fdp(func))
            self.approxRDP = pointwise_minimum_two_arguments(self.approxRDP,
                                 approxdp_func_to_approxrdp(func))
            self.approxDP = pointwise_minimum(self.approxDP, func)
            
        elif type_of_update == 'RDP':
            # function output RDP eps as a function of alpha
            self.RenyiDP = pointwise_minimum(self.RenyiDP, func)
            self.approxDP = pointwise_minimum(self.approxDP,
                     rdp_to_approxdp(self.RenyiDP, BBGHS_conversion=BBGHS_conversion))
            self.fDP = pointwise_maximum(self.fDP,
                                                   approxdp_func_to_fdp(
                                                       self.approxDP))
        else:
            print(type_of_update, ' not recognized.')

#### Our PySyft classes begin from here on

In [9]:
# returns the privacy budget spent by each entity
@lru_cache(maxsize=None)
def _individual_RDP_gaussian(
    sigma: float, value: float, L: float, alpha: float
) -> float:
    return (alpha * (L**2) * (value**2)) / (2 * (sigma**2))

In [10]:
def individual_RDP_gaussian(params: Dict, alpha: float) -> np.float64:
    """
    :param params:
        'sigma' --- is the normalized noise level: std divided by global L2 sensitivity
        'value' --- is the output of query on a data point
        'L' --- is the Lipschitz constant of query with respect to the output of query on a data point
    :param alpha: The order of the Renyi Divergence
    :return: Evaluation of the RDP's epsilon
    """
    sigma = params["sigma"]
    value = params["value"]
    L = params["L"]
    if sigma <= 0:
        raise Exception("Sigma should be above 0")
    if alpha < 0:
        raise Exception("Sigma should not be below 0")

    return _individual_RDP_gaussian(sigma=sigma, alpha=alpha, value=value, L=L)

In [11]:
# Example of a specific mechanism that inherits the Mechanism class
# @serializable(recursive_serde=True)
class iDPGaussianMechanism(Mechanism):
    __attr_allowlist__ = [
        "name",
        "params",
        "entity_name",
        "delta0",
        "RDP_off",
        "approxDP_off",
        "user_key",
    ]

    def __init__(
        self,
        sigma: float,
        squared_l2_norm: float,
        squared_l2_norm_upper_bound: float,
        L: float,
        entity_name: str,
        name: str = "Gaussian",
        RDP: bool = True,
        approxDP: bool = True,
        user_key: Optional[VerifyKey] = None,  #TODO: Why isn't it mandatory to provide a User Key?
    ):

        Mechanism.__init__(self)

        self.user_key = user_key

        self.name = name  # When composing
        self.params = {
            "sigma": float(sigma),
            "private_value": float(squared_l2_norm),
            "public_value": float(squared_l2_norm_upper_bound),
            "L": float(L),
        }  # This will be useful for the Calibrator

        self.entity_name = entity_name

        self.delta0 = 0
        if RDP:
            # Tudor: i'll fix these  
            # x is the alpha value of the RDP here
            new_rdp = lambda x: individual_RDP_gaussian(self.params, x)  # noqa: E731
            self.propagate_updates(new_rdp, "RDP")

        if approxDP:  # Direct implementation of approxDP
            new_approxdp = lambda x: dp_bank.get_eps_ana_gaussian(  # noqa: E731
                sigma, x
            )
            self.propagate_updates(new_approxdp, "approxDP_func")

In [12]:
m = Mechanism()

In [13]:
idpgm = iDPGaussianMechanism(sigma=1, squared_l2_norm=40, squared_l2_norm_upper_bound=10, L=5, entity_name="Bob")

In [14]:
idpgm.delta0

0

#### This part was in the adversarial accountant

In [15]:
class Transformer():
    """
    A transformer is a callable object that takes one or more mechanism as input and
    **transform** them into a new mechanism
    """

    def __init__(self):
        self.name = 'generic_transformer'
        self.unary_operator = False  # If true it takes one mechanism as an input,
        # otherwise it could take many, e.g., composition
        self.preprocessing = False  # Relevant if unary is true, it specifies whether the operation
        # is before or after the mechanism, e.g., amplification by sampling is before applying the
        # mechanism, "amplification by shuffling" is after applying the LDP mechanisms
        self.transform = lambda x: x

    def __call__(self, *args, **kwargs):
        return self.transform(*args, **kwargs)

In [16]:
t = Transformer()

In [26]:
import math

# The generic composition class
class Composition(Transformer):
    """ Composition is a transformer that takes a list of Mechanisms and number of times they appear,
    and output a Mechanism that represents the composed mechanism"""
    def __init__(self):
        Transformer.__init__(self)
        self.name = 'Composition'

        # Update the function that is callable
        self.transform = self.compose

    def compose(self, mechanism_list, coeff_list):
        # Make sure that the mechanism has a unique list
        # for example, if there are two Gaussian mechanism with two different sigmas, call it
        # Gaussian1, and Gaussian2


        newmech = Mechanism()

        # update the functions
        def newrdp(x):
            return sum([c * mech.RenyiDP(x) for (mech, c) in zip(mechanism_list, coeff_list)])
        newmech.propagate_updates(newrdp, 'RDP')

        # TODO: the fDP_based_conversion sometimes fails due to undefined RDP with alpha < 1

        newmech.eps_pureDP = sum([c * mech.eps_pureDP for (mech, c)
                                  in zip(mechanism_list, coeff_list)])
        newmech.delta0 = max([mech.delta0 for (mech, c)
                              in zip(mechanism_list, coeff_list)])

        # Other book keeping
        newmech.name = self.update_name(mechanism_list, coeff_list)
        # keep track of all parameters of the composed mechanisms
        newmech.params = self.update_params(mechanism_list)

        return newmech

    def update_name(self,mechanism_list, coeff_list):
        separator = ', '
        s = separator.join([mech.name + ': ' + str(c) for (mech, c)
                           in zip(mechanism_list, coeff_list)])

        return 'Compose:{'+ s +'}'

    def update_params(self, mechanism_list):
        params = {}
        for mech in mechanism_list:
            params_cur = {mech.name+':'+k: v for k,v in mech.params.items()}
            params.update(params_cur)
        return params

In [27]:
c = Composition()
m = c.compose([idpgm], [1])

In [29]:
def compose_mechanisms(
    mechanisms: Iterable[iDPGaussianMechanism], delta: float
) -> float:
    sigmas = list()
    squared_l2_norms = list()
    squared_l2_norm_upper_bounds = list()
    Ls = list()
    values = list()

    for m in mechanisms:
        sigmas.append(m.params["sigma"])
        squared_l2_norms.append(m.params["private_value"])
        squared_l2_norm_upper_bounds.append(m.params["public_value"])
        Ls.append(m.params["L"])
        values.append(m.params["value"])

    return compose_mechanisms_via_simplified_args_for_lru_cache(
        tuple(sigmas),
        tuple(squared_l2_norms),
        tuple(squared_l2_norm_upper_bounds),
        tuple(Ls),
        tuple(values),
        delta,
    )


@lru_cache(maxsize=None)
def compose_mechanisms_via_simplified_args_for_lru_cache(
    sigmas: Tuple[float],
    squared_l2_norms: Tuple[float],
    squared_l2_norm_upper_bounds: Tuple[float],
    Ls: Tuple[float],
    values: Tuple[float],
    delta: float,
) -> float:
    mechanisms = list()
    for i in range(len(sigmas)):

        m = iDPGaussianMechanism(
            sigma=sigmas[i],
            squared_l2_norm=squared_l2_norms[i],
            squared_l2_norm_upper_bound=squared_l2_norm_upper_bounds[i],
            L=Ls[i],
            entity_name="",
        )
        m.params["value"] = values[i]
        mechanisms.append(m)
    # compose them with the transformation: compose
    compose = Composition()
    composed_mech = compose(mechanisms, [1] * len(mechanisms))
    eps = composed_mech.get_approxDP(delta)
    return eps


In [30]:
idpgm

In [50]:
inputs = []

for i in range(10**6):
    inputs.append(iDPGaussianMechanism(sigma=2, squared_l2_norm=5 + i, squared_l2_norm_upper_bound=7 + i, L=4, entity_name=str(i)))
    inputs[i].params["value"] = inputs[i].params["private_value"]

In [51]:
%%time
compose_mechanisms(inputs, delta=1e-6)

CPU times: user 2min 47s, sys: 4.18 s, total: 2min 52s
Wall time: 2min 51s


6.66675677715008e+17

In [52]:
inputs = []

for i in range(10**6):
    inputs.append(iDPGaussianMechanism(sigma=2, squared_l2_norm=50, squared_l2_norm_upper_bound=7, L=4, entity_name=str(i)))
    inputs[i].params["value"] = inputs[i].params["private_value"]

In [53]:
%%time
compose_mechanisms(inputs, delta=1e-6)

CPU times: user 1min 57s, sys: 1.18 s, total: 1min 59s
Wall time: 1min 59s


5000525641.237593